# Homework 4: Academy of Py

#### *Grouping the data by spending, I can tell that the overall passing rate is lowest at schools with the highest per student budget.  
#### *The overall pass rate is much higher at charter schools than district schools.  In particular, the average reading score is almost 20 percentage points higher at charter schools
#### *The overall pass rate is much lower at the largest schools 

In [26]:
#import dependencies
import pandas as pd
import numpy as np

In [38]:
#getting our data
csv_path_1 = "./students_complete.csv"
student_data_df = pd.read_csv(csv_path_1)
student_data_df.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score'],
      dtype='object')

In [28]:
csv_path = "./schools_complete.csv"
survey_data_df = pd.read_csv(csv_path)
survey_data_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## District Summary

In [29]:
district_summary={"totalSchools":"","totalStudents":"",
                  "totalBudget":"","avgMath":"","avgRead":"",
                  "mathPer":"","readPer":"","overallPer":""}

#finds the total number of schools
total_s=survey_data_df.shape
total_schools=total_s[0]
district_summary["totalSchools"]=total_schools
#finds the total number of students
total_st=student_data_df.shape
total_students=total_st[0]
district_summary["totalStudents"]=total_students

In [30]:
#finds the total budget
total_budget=survey_data_df["budget"].sum()
district_summary["totalBudget"]=total_budget
#finds the average reading and math scores
average_reading=student_data_df["reading_score"].mean()
average_math=student_data_df["math_score"].mean()
district_summary["avgMath"]=round(average_math,1)
district_summary["avgRead"]=round(average_reading,1)

In [31]:
#finds the percentage of students who passed reading
read_passers=student_data_df.loc[student_data_df['reading_score'] >= 70]
num_read_p=read_passers.shape
num_read_passers=num_read_p[0]
read_pass_per=float((num_read_passers/total_students)*100)
read_pass_per_neat=("{0:.1f}%".format(read_pass_per))
district_summary["readPer"]=read_pass_per_neat

In [32]:
#finds the percentage of students who passed math
math_passers=student_data_df.loc[student_data_df['math_score'] >= 70]
num_math_p=math_passers.shape
num_math_passers=num_math_p[0]
math_pass_per=float((num_math_passers/total_students)*100)
math_pass_per_neat=("{0:.1f}%".format(math_pass_per))
district_summary["mathPer"]=math_pass_per_neat

In [33]:
#Averages the % of math and reading passers
overall_pass=(math_pass_per+read_pass_per)/2
overall_pass_neat=("{0:.1f}%".format(overall_pass))
district_summary["overallPer"]=overall_pass_neat

In [34]:
district_summary_df=pd.DataFrame(district_summary,index=[1])
district_summary_df=district_summary_df[["totalSchools","totalStudents","totalBudget",
                                         "avgMath","avgRead","mathPer","readPer","overallPer"]]
district_summary_df.columns=['Total Schools', 'Total Students', 
                            'Total Budget', 'Avg Math Score','Avg Reading Score','% Passing Math',
                            '% Passing Reading', 'Overall Passing Rate', 
                            ]
district_summary_df

,Total Schools,Total Students,Total Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
1,15,39170,24649428,79.0,81.9,75.0%,85.8%,80.4%


## School Summary

In [35]:
#make a function to get the data I need for each school
def extractStudents(schoolName):
    school_overall_pass_neat=0
    school_read_per=0
    school_average_math=0
    school_avg_math=0
    school_avg_read=0
    school_budget=0
    school_math_passers=0
    school_math_per=0
    school_math_per_neat=0
    school_dic={'name':schoolName, 'type':'','numStudents':'',
                'totalBudget':'','stuBudget':'','mathScore':'',
                'readScore':'', 'mathPer':'','readPer':'',
                'overallPer':''}
    
    #create a Data Frame for the given school
    school_name_df=student_data_df.loc[student_data_df["school"] == schoolName]
    overall_school_name_df=survey_data_df.loc[survey_data_df["school"]==schoolName]
    #finds the school type, size, budget and per student budget
    school_type=overall_school_name_df['type'] 
    if 'Charter' in str(school_type):
        school_type="Charter"
    else:
        school_type="District"
    school_budget=overall_school_name_df['budget']
    school_size=overall_school_name_df['size']
    per_student_budget=int(school_budget)/int(school_size)
    
    school_dic['type']=school_type
    school_dic['numStudents']=int(school_size)
    school_dic['totalBudget']=int(school_budget)
    school_dic['stuBudget']=per_student_budget

    #finds the percentage of students who passed math and the average math score
    school_math_passers_df=school_name_df.loc[student_data_df['math_score'] >= 70]
    school_math_passers=school_math_passers_df.shape[0]
    school_math_per=(int(school_math_passers)/int(school_size))*100
    school_math_per_neat=("{0:.2f}%".format(school_math_per))
    school_avg_math=school_name_df["math_score"].mean()
    school_dic['mathScore']=round(school_avg_math,2)  
    school_dic['mathPer']=round(school_math_per,2)  #Note that this isn't a number!!!!!
    
    #finds the percentage of students who passed reading and the average reading score
    school_read_passers_df=school_name_df.loc[student_data_df['reading_score'] >= 70]
    school_read_passers=school_read_passers_df.shape[0]
    school_read_per=(int(school_read_passers)/int(school_size))*100
    school_read_per_neat=("{0:.2f}%".format(school_read_per)) 
    school_avg_read=school_name_df["reading_score"].mean()
    school_dic['readScore']=round(school_avg_read,2)
    school_dic['readPer']=round(school_read_per,2)

    #Averages the % of math and reading passers
    school_overall_pass=round((school_math_per+school_read_per)/2,2)
    school_overall_pass_neat=("{0:.2f}%".format(school_overall_pass))

    school_overall_pass_neat=("{0:.2f}%".format(school_overall_pass))
    school_dic['overallPer']=school_overall_pass
   
    return school_dic

In [40]:
huang_dictionary=extractStudents('Huang High School')
figueroa_dictionary=extractStudents('Figueroa High School')
shelton_dictionary=extractStudents('Shelton High School')
hernandez_dictionary=extractStudents('Hernandez High School')
griffin_dictionary=extractStudents('Griffin High School')
wilson_dictionary=extractStudents('Wilson High School')
cabrera_dictionary=extractStudents('Cabrera High School')
bailey_dictionary=extractStudents('Bailey High School')
holden_dictionary=extractStudents('Holden High School')
pena_dictionary=extractStudents('Pena High School')
wright_dictionary=extractStudents('Wright High School')
rodriguez_dictionary=extractStudents('Rodriguez High School')
johnson_dictionary=extractStudents('Johnson High School')
gomez_dictionary=extractStudents('Bailey High School')
ford_dictionary=extractStudents('Ford High School')
thomas_dictionary=extractStudents('Thomas High School')

KeyError: 'school'

In [14]:
all_schools=[huang_dictionary, figueroa_dictionary,
            shelton_dictionary, hernandez_dictionary,
             griffin_dictionary, wilson_dictionary,
            cabrera_dictionary, bailey_dictionary,
            holden_dictionary, pena_dictionary,
            wright_dictionary, rodriguez_dictionary,
            johnson_dictionary, gomez_dictionary,
            ford_dictionary, thomas_dictionary,]
all_schools_df=pd.DataFrame(all_schools)
all_schools_df_neat=all_schools_df[["name","type","numStudents",
                                    "totalBudget","stuBudget",
                                    "mathScore","readScore","mathPer",
                                    "readPer","overallPer"]]
all_schools_df_neat.columns=['School Name', 'School Type', 
                            '# of Students', 'Total School Budget',
                             'Per Student Budget','Avg Math Score',
                             'Avg Reading Score','% Passing Math',
                            '% Passing Reading', 'Overall Passing Rate (%)', 
                            ]
all_schools_df_neat.head(11)

NameError: name 'miller_dictionary' is not defined

## Top 5 Schools by Passing Rate

In [571]:
#finds the top five performing schools by overall passing rate
top_5_df=all_schools_df_neat.nlargest(5,"Overall Passing Rate (%)")
top_5_df

,School Name,School Type,# of Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
9,Campbell High School,Charter,271,157993,583.0,83.59,93.77,95.20,100.0,97.60
10,Macdonald High School,Charter,901,550511,611.0,83.78,93.93,94.34,100.0,97.17
2,Galloway High School,Charter,2471,1445535,585.0,83.57,94.03,94.21,100.0,97.11
8,Gomez High School,Charter,2154,1288092,598.0,83.84,94.03,93.96,100.0,96.98
0,Miller High School,Charter,2424,1418040,585.0,83.61,94.00,93.56,100.0,96.78


## Bottom 5 Schools by Passing Rate

In [595]:
#finds the bottom five performing schools by overall passing rate
bottom_5_df=all_schools_df_neat.nsmallest(5,"Overall Passing Rate (%)")
bottom_5_df

,School Name,School Type,# of Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate (%)
3,Smith High School,District,4954,3210192,648.0,81.54,77.15,83.17,66.37,74.77
7,Glass High School,District,3271,2155589,659.0,81.29,76.89,82.33,67.35,74.84
4,Kelly High School,District,3307,2225611,673.0,81.68,76.83,83.40,66.56,74.98
1,Sherman High School,District,3213,2152710,670.0,81.50,77.29,82.94,67.29,75.12
5,Hawkins High School,District,4555,2851430,626.0,81.72,77.01,83.91,66.92,75.41


## Math Scores By Grade

In [573]:
merge_table = pd.merge(student_data_df, survey_data_df, on="school_name")
grouped_df_new=merge_table['math_score'].groupby([merge_table['school_name'],merge_table['grade']])
mean_grouped_new=grouped_df_new.mean()
mean_grouped_new=mean_grouped_new.unstack()
mean_grouped_new1=mean_grouped_new[['9th','10th','11th','12th']]
mean_grouped_new1["9th"] = mean_grouped_new1["9th"].map("{:.1f}%".format)
mean_grouped_new1["10th"] = mean_grouped_new1["10th"].map("{:.1f}%".format)
mean_grouped_new1["11th"] = mean_grouped_new1["11th"].map("{:.1f}%".format)
mean_grouped_new1["12th"] = mean_grouped_new1["12th"].map("{:.1f}%".format)
mean_grouped_new1

grade,9th,10th,11th,12th
school_name,,,,
Campbell High School,83.8%,84.3%,83.9%,82.1%
Galloway High School,83.5%,83.6%,84.0%,83.2%
Glass High School,81.9%,81.0%,81.4%,80.8%
Gomez High School,83.7%,84.0%,83.9%,83.8%
Gonzalez High School,83.5%,84.0%,83.2%,82.8%
Hawkins High School,81.7%,81.5%,81.9%,81.9%
Kelly High School,81.8%,81.9%,81.5%,81.5%
Macdonald High School,84.3%,83.8%,83.5%,83.5%
Miller High School,83.8%,83.6%,83.6%,83.3%


## Reading Scores By Grade

In [574]:
read_table=merge_table['reading_score'].groupby([merge_table['school_name'],merge_table['grade']]).mean()
read_table_happy=read_table.unstack()
read_table_happy1=read_table_happy[['9th','10th','11th','12th']]
read_table_happy1["9th"] = read_table_happy1["9th"].map("{:.1f}%".format)
read_table_happy1["10th"] = read_table_happy1["10th"].map("{:.1f}%".format)
read_table_happy1["11th"] = read_table_happy1["11th"].map("{:.1f}%".format)
read_table_happy1["12th"] = read_table_happy1["12th"].map("{:.1f}%".format)
read_table_happy1

grade,9th,10th,11th,12th
school_name,,,,
Campbell High School,93.5%,93.9%,94.1%,93.7%
Galloway High School,94.1%,94.0%,94.0%,94.1%
Glass High School,76.4%,77.3%,77.1%,76.6%
Gomez High School,94.2%,94.0%,93.8%,94.1%
Gonzalez High School,94.0%,94.1%,94.4%,94.0%
Hawkins High School,76.5%,77.2%,77.5%,76.9%
Kelly High School,76.4%,77.3%,76.6%,77.0%
Macdonald High School,94.0%,94.1%,93.8%,93.7%
Miller High School,93.9%,94.0%,94.2%,93.8%


## Scores By Per Student Spending

In [575]:
#grouping schools by per student spending
bins = [0, 585, 615, 645, 700]

# Create the names for the four bins
group_names = ['$500-585', '$585-615', '$615-645', '$645-700']

all_schools_df["Per Student Budget Summary"] = pd.cut(all_schools_df["stuBudget"], bins, labels=group_names)

all_schools_df_new = all_schools_df[['name','mathScore','readScore','mathPer','readPer',
                                     'overallPer','Per Student Budget Summary']]

all_schools_df_new.columns=['School Name', 'Avg Math Score', 
                            'Avg Reading Score', '% Passing Math',
                            '% Passing Reading', 'Overall Passing Rate', 
                            'Per Student Budget Summary']
spend_table=all_schools_df_new.groupby('Per Student Budget Summary')
spend_table_happy=spend_table.mean()
spend_table_happy["% Passing Math"] = spend_table_happy["% Passing Math"].map("{:.1f}%".format)
spend_table_happy["% Passing Reading"] = spend_table_happy["% Passing Reading"].map("{:.1f}%".format)
spend_table_happy["Overall Passing Rate"] = spend_table_happy["Overall Passing Rate"].map("{:.1f}%".format)
spend_table_happy["Avg Math Score"] = spend_table_happy["Avg Math Score"].map("{:.1f}".format)
spend_table_happy["Avg Reading Score"] = spend_table_happy["Avg Reading Score"].map("{:.1f}".format)
spend_table_happy

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Per Student Budget Summary,,,,,
$500-585,83.6,93.9,94.3%,100.0%,97.2%
$585-615,83.8,94.0,94.2%,100.0%,97.1%
$615-645,82.6,85.6,88.6%,83.5%,86.0%
$645-700,81.5,77.0,83.0%,66.9%,74.9%


## Grouping Schools By Size

In [576]:
bins = [0, 1500, 3000, 5000]
#groups = all_schools_df.groupby(['size', pd.cut(all_schools_df.views, bins)])

In [577]:
#grouping schools by size
bins = [0, 1500, 3000, 5000]

# Create the names for the four bins
size_group_names = ['Small (<1,500)', 'Medium (1,500-3,000)', 'Large (3,000-5,000)']

all_schools_df["Size Summary"] = pd.cut(all_schools_df["numStudents"], bins, labels=size_group_names)
new_column_names={'name':"School Name", 'mathScore':"Avg Math Score", 
                  'readScore':"Avg Reading Score",'mathPer':"% Passing Math",
                 'readPer':"% Passing Reading", 'overallPer':"Overall Passing Rate"}
all_schools_df_new_size = all_schools_df[['name','mathScore','readScore','mathPer','readPer','overallPer','Size Summary']]
all_schools_df_new_size.rename(columns=new_column_names,inplace=True)
size_table=all_schools_df_new_size.groupby('Size Summary')
size_table_happy=size_table.mean()
size_table_happy["% Passing Math"] = size_table_happy["% Passing Math"].map("{:.1f}%".format)
size_table_happy["% Passing Reading"] = size_table_happy["% Passing Reading"].map("{:.1f}%".format)
size_table_happy["Overall Passing Rate"] = size_table_happy["Overall Passing Rate"].map("{:.1f}%".format)
size_table_happy["Avg Math Score"] = size_table_happy["Avg Math Score"].map("{:.1f}".format)
size_table_happy["Avg Reading Score"] = size_table_happy["Avg Reading Score"].map("{:.1f}".format)
size_table_happy



C:\Users\Julie\Anaconda4\lib\site-packages\pandas\core\frame.py:2844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Size Summary,,,,,
"Large (3,000-5,000)",81.5,77.0,83.2%,66.9%,75.0%
"Medium (1,500-3,000)",83.6,94.1,93.7%,100.0%,96.9%
"Small (<1,500)",83.7,93.8,94.8%,100.0%,97.4%


## Grouping Schools by Type

In [578]:
type_table=all_schools_df.groupby(all_schools_df['type'])

type_table=type_table[['mathScore','readScore', 'mathPer', 'readPer', 'overallPer']]
type_table_mean=type_table.mean()
type_table_mean.columns=['Avg Math Score', 
                            'Avg Reading Score', '% Passing Math',
                            '% Passing Reading', 'Overall Passing Rate', 
                            ]
type_table_mean["% Passing Math"] = type_table_mean["% Passing Math"].map("{:.1f}%".format)
type_table_mean["% Passing Reading"] = type_table_mean["% Passing Reading"].map("{:.1f}%".format)
type_table_mean["Overall Passing Rate"] = type_table_mean["Overall Passing Rate"].map("{:.1f}%".format)
type_table_mean["Avg Math Score"] = type_table_mean["Avg Math Score"].map("{:.1f}".format)
type_table_mean["Avg Reading Score"] = type_table_mean["Avg Reading Score"].map("{:.1f}".format)
type_table_mean.head()

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.6,94.0,94.1%,100.0%,97.0%
District,81.5,77.0,83.2%,66.9%,75.0%
